# Анализ финансовых транзакций и клиентов

## Составление DataFrame

Воспользуемся Pandas: чтобы из clients_data.json и transactions_data.xlsx получить датасеты и работать в последствии именно с ними!

И посмотрим, конечно, что все действительно выгрузилось! (таблички объемные, поэтому ограничим вывод)

In [2]:
import pandas as pd

clients_data = pd.read_json('./data/clients_data.json')
clients_data.head()

,id,age,gender,net_worth
0,1ec5ab91-fdd0-4925-a221-f0c4e04fe0e7,41.0,Мужчина,2514729.46
1,c940f45c-d29b-439a-8fa3-f75018cef528,28.0,Мужчина,566568.29
2,cfa21d7e-8499-43d0-9dfb-f4a1b94ce44c,55.0,Женщина,1896001.28
3,NaN,38.0,Женщина,4538759.60
4,3f43d7bc-37f0-4535-8391-51702d6ff81e,31.0,Мужчина,4865849.92


In [9]:
transactions_data = pd.read_excel('./data/transactions_data.xlsx')
transactions_data.head()

,transaction_id,client_id,transaction_date,service,amount,payment_method,city,consultant
0,ff36597c-df41-44a0-9f46-6aa1fe615270,1ec5ab91-fdd0-4925-a221-f0c4e04fe0e7,2025-01-04 00:02:20,Структурирование капитала,55838.059789,Банковский перевод,North Patrickport,Cheryl Waller
1,0d5263ec-413d-44d9-a9d3-a996e3a56b05,c940f45c-d29b-439a-8fa3-f75018cef528,2025-03-10 17:16:50,Структурирование капитала,37514.809209,Неизвестно,New Zacharyport,Frank Pollard
2,5e9a8978-df1c-483f-a53b-16149c93f11e,cfa21d7e-8499-43d0-9dfb-f4a1b94ce44c,2025-02-13 19:43:49,Структурирование капитала,21022.146872,Неизвестно,Port Michellemouth,Alexandra Meyer
3,524031ea-56e2-410a-aee2-e8fc6ae3fbc5,NaN,2025-03-02 03:14:10,Финансовое планирование,2313.980219,Неизвестно,Erichaven,Patricia Haas
4,421aaf04-c97d-42ac-b35f-09a56bbaf910,3f43d7bc-37f0-4535-8391-51702d6ff81e,2025-03-18 21:00:36,Структурирование капитала,5636.586955,Неизвестно,Bondstad,Melissa Pena


## Проверка данных у клиентов

In [44]:
clients_data.info()

print("\n Дубликатов строк: ", clients_data.duplicated().sum())
print("\n Дубликатов id: ", clients_data['id'].duplicated().sum())

print("\n Минимальный возраст: ", clients_data['age'].min())
print("\n Максимальный возраст: ", clients_data['age'].max())
print("\n Минимальная стоимость активов клиента: ", clients_data['net_worth'].min())

print("\n Смотрим, где есть NaN!")
print(clients_data.isnull().sum())

<class 'pandas.DataFrame'>
RangeIndex: 9799 entries, 0 to 9798
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         9798 non-null   str    
 1   age        8816 non-null   float64
 2   gender     6480 non-null   str    
 3   net_worth  9324 non-null   float64
dtypes: float64(2), str(2)
memory usage: 306.3 KB

 Дубликатов строк:  0

 Дубликатов id:  0

 Минимальный возраст:  20.0

 Максимальный возраст:  60.0

 Минимальная стоимость активов клиента:  10213.97

 Смотрим, где есть NaN!
id              1
age           983
gender       3319
net_worth     475
dtype: int64


## Проверка данных в таблице с транзакциями

In [53]:
transactions_data.info()

print("\n Дубликатов строк: ", transactions_data.duplicated().sum())
print("\n Дубликатов id транзакций: ", transactions_data['transaction_id'].duplicated().sum())

print("\n Минимальное значение транзакции: ", transactions_data['amount'].min())
print("\n Невалидные даты: ", (transactions_data['transaction_date'] == 'INVALID_DATE').sum())

print("\n Смотрим, где есть NaN!")
print(transactions_data.isnull().sum())

print("\n Какие есть уникальные значения в каждом поле (кроме id клиентов и транзакций)? Цель - найти что-то неизвестное\n")
print("\n Услуги:", transactions_data['service'].unique())
print("\n Способы оплаты:", transactions_data['payment_method'].unique())
print("\n Города:", transactions_data['city'].unique())
print("\n Консультанты:", transactions_data['consultant'].unique())

<class 'pandas.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transaction_id    9526 non-null   str    
 1   client_id         9798 non-null   str    
 2   transaction_date  10000 non-null  object 
 3   service           10000 non-null  str    
 4   amount            9540 non-null   float64
 5   payment_method    10000 non-null  str    
 6   city              10000 non-null  str    
 7   consultant        10000 non-null  str    
dtypes: float64(1), object(1), str(6)
memory usage: 625.1+ KB

 Дубликатов строк:  0

 Дубликатов id транзакций:  473

 Минимальное значение транзакции:  14.04490704660066

 Невалидные даты:  1005

 Смотрим, где есть NaN!
transaction_id      474
client_id           202
transaction_date      0
service               0
amount              460
payment_method        0
city                  0
consultant            0
dtype: int64

 Какие

In [10]:
duplicated_ids = transactions_data.loc[transactions_data['transaction_id'].duplicated(keep=False), 'transaction_id']
print(duplicated_ids)

11      NaN
51      NaN
177     NaN
205     NaN
295     NaN
       ... 
9917    NaN
9928    NaN
9937    NaN
9950    NaN
9967    NaN
Name: transaction_id, Length: 474, dtype: str


## Какие проблемы в данных?

### Клиенты

- Неуникальных id нет!
- Поле не заполнено: NaN в id, age, gender, net_worth

- Невалидные значения (необнаруженные:)):

    - Невалидный возраст (например, меньше 0 лет), но минимальный возраст 20.0, максимальный - 60.0
    - Отрицательные значения в столбце net_worth (так как в условии в качестве примера аномалии указаны отрицательные значения транзакций), но минимальное значение в этом столбце 10214

Итог:
- необходимо удалить строчки, в которых есть пропущенные данные (NaN !!кроме гендера!!)
- хотелось бы не потерять еще одного клиента с неуказанным id, но транзакции привязаны к его id, поэтому придется удалить
- пол заменим на строку, что гендер не указан

### Транзакции

- Дубликаты в transaction_id (вероятнее всего это все NaN)

- Поле не заполнено:

    - NaN в transaction_id, client_id, amount

    В других столбцах NaN нет, но:
    - Неизвестная услуга в service
    - Неизвестный способ оплаты в payment_method
    - Неизвестный город в city
    - Неизвестный консультант в consultant

    Будто, это не так страшно, так как можно было бы выделить в отдельную группу, но с учетом того, что в дальнейшем я планирую прогнозировать спрос на определенную услугу (а можно было бы усложнить и посмотреть, какое количество клиентов может быть у определенного консультанта), а также определить топ-5 наиболее популярных услуг по количеству заказов, рассчитать среднюю сумму транзакций по каждому городу, определить услугу с наибольшей выручкой, вычислить процент транзакций по способам оплаты, то такие большие группы с неизвестными данными помешают эффективно оценить, поэтому такие строчки тоже удалим

- Невалидные значения:

    - INVALID_DATE в transaction_date
    - Отрицательных транзакций нет (минимальная 14)


Итог:
- удаляем строки в которых поля содержат либо NaN, либо какие-либо неизвестные данные
- приводим к единому формату даты транзакций
- заменим повторяющиеся id 